# Streamlit LLM Translator — Multi‑provider
Translate text between languages using OpenAI, Gemini, Anthropic, Groq, or Hugging Face models with a simple Streamlit UI.


# Installation (commented)

In [ ]:
# !pip install streamlit openai anthropic google-generativeai groq huggingface_hub


# Imports and helpers

In [ ]:
import os
import streamlit as st

# Provider factories (lazy)
def _openai():
    from openai import OpenAI
    if (k:=st.secrets.get('OPENAI_API_KEY', None) if hasattr(st,'secrets') else None) or os.environ.get('OPENAI_API_KEY'):
        os.environ['OPENAI_API_KEY'] = k or os.environ.get('OPENAI_API_KEY','')
    return OpenAI()

def _gemini(model_name):
    import google.generativeai as genai
    genai.configure(api_key=st.secrets.get('GEMINI_API_KEY', os.environ.get('GEMINI_API_KEY')))
    return genai.GenerativeModel(model_name)

def _anthropic():
    import anthropic
    if (k:=st.secrets.get('ANTHROPIC_API_KEY', None) if hasattr(st,'secrets') else None) or os.environ.get('ANTHROPIC_API_KEY'):
        os.environ['ANTHROPIC_API_KEY'] = k or os.environ.get('ANTHROPIC_API_KEY','')
    return anthropic.Anthropic()

def _groq():
    from groq import Groq
    return Groq(api_key=st.secrets.get('GROQ_API_KEY', os.environ.get('GROQ_API_KEY')))

def _hf():
    from huggingface_hub import InferenceClient
    return InferenceClient(token=st.secrets.get('HUGGINGFACEHUB_API_TOKEN', os.environ.get('HUGGINGFACEHUB_API_TOKEN')))


# UI

In [ ]:
st.set_page_config(page_title="LLM Translator", page_icon="🌐")
st.title("🌐 LLM Translator — Multi‑provider")

with st.sidebar:
    provider = st.selectbox("Provider", ["OpenAI","Gemini","Anthropic","Groq","HuggingFace"], index=0)
    temperature = st.slider("temperature", 0.0, 1.5, 0.3, 0.1)
    max_tokens = st.slider("max tokens", 64, 2048, 512, 32)

    if provider=="OpenAI":
        model = st.text_input("Model", value="gpt-4o-mini")
    elif provider=="Gemini":
        model = st.text_input("Model", value="gemini-1.5-flash")
    elif provider=="Anthropic":
        model = st.text_input("Model", value="claude-3-5-sonnet-20241022")
    elif provider=="Groq":
        model = st.text_input("Model", value="llama-3.1-8b-instant")
    else:
        model = st.text_input("Model", value="Helsinki-NLP/opus-mt-en-fr")

src = st.selectbox("Source language", ["auto","English","French","Spanish","German","Hindi","Chinese","Japanese"])
dst = st.selectbox("Target language", ["English","French","Spanish","German","Hindi","Chinese","Japanese"]) 

text = st.text_area("Text to translate", height=160, placeholder="Paste text here…")


# Translate function

In [ ]:
def translate(provider, model, src, dst, text):
    instruction = f"Translate the following text from {src} to {dst}. Keep meaning and tone. Only output the translation."
    if provider=="OpenAI":
        client = _openai()
        msgs = [{"role":"system","content":instruction},{"role":"user","content":text}]
        out = client.chat.completions.create(model=model, messages=msgs, temperature=temperature, max_tokens=max_tokens)
        return out.choices[0].message.content
    if provider=="Gemini":
        g = _gemini(model)
        resp = g.generate_content(f"{instruction}\n\n{text}")
        return getattr(resp,'text',str(resp))
    if provider=="Anthropic":
        a = _anthropic()
        resp = a.messages.create(model=model, max_tokens=max_tokens, temperature=temperature,
                                 system=instruction, messages=[{"role":"user","content":text}])
        return resp.content[0].text
    if provider=="Groq":
        gq = _groq()
        msgs = [{"role":"system","content":instruction},{"role":"user","content":text}]
        resp = gq.chat.completions.create(model=model, messages=msgs, temperature=temperature, max_tokens=max_tokens)
        return resp.choices[0].message.content
    if provider=="HuggingFace":
        hf = _hf()
        try:
            msgs = [{"role":"system","content":instruction},{"role":"user","content":text}]
            resp = hf.chat_completion(model=model, messages=msgs, max_tokens=max_tokens, temperature=temperature)
            return resp.choices[0].message['content'] if hasattr(resp.choices[0],'message') else resp.choices[0]['message']['content']
        except Exception:
            prompt = f"{instruction}\n\n{text}"
            return hf.text_generation(model=model, inputs=prompt, max_new_tokens=max_tokens, temperature=temperature)
    raise ValueError("Unsupported provider")


# Run

In [ ]:
if st.button("Translate", type="primary") and text.strip():
    try:
        out = translate(provider, model, src, dst, text)
    except Exception as e:
        out = f"Error: {e}"
    st.subheader("Translation")
    st.code(out)


# Notes
# - For Hugging Face, you can also select specialized translation models per language pair.
# - For bulk translation, paste multiple lines; you can add splitting and CSV download easily.